In [ ]:
from pasteur.kedro.ipython import *
register_kedro()

In [ ]:
%pipe tab_adult.ingest
for e1 in (0.3, 0.5, 0.9, 1.5):
    pipe("tab_adult.privbayes.synth", {"alg.e1": e1})

2000-01-01 00:00:00,000 - pasteur.kedro.runner - INFO - Executing pipeline tab_adult.ingest
Executing tab_adult.wrk.transform_table: 100%|██████████| 10/10 [00:00<00:00, 22.62it/s]
2000-01-01 00:00:00,000 - pasteur.kedro.runner - INFO - Executing pipeline tab_adult.privbayes.synth with overrides `alg.e1=0.3`
2000-01-01 00:00:00,000 - pasteur.synth.privbayes - WARNING - Current variable domain implementation (required by ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']) technically violates DP (to be fixed).
Executing tab_adult.privbayes.fit:   0%|          | 0/3 [00:00<?, ?it/s]

























Finding Nodes: 100%|██████████| 13/13 [00:00<00:00, 73.20it/s]
2000-01-01 00:00:00,000 - pasteur.synth.privbayes - INFO - Bayesian Network Tree:
(PrivBayes e1=0.50, e2=1.60, theta=2.00, available t=372.11)
┌─────────────────────┬─────┬──────────┬──────────────────────────────────────────────────────────────────────┐
│           Attribute │ Dom │ Avail